In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import requests
import time
from tqdm import tqdm
import os
import re
import datetime

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException # 검색 결과가 없는 경우 예외 처리
from bs4 import BeautifulSoup

In [3]:
# 수집 날짜 지정(전일자)
before_one_day = datetime.datetime.now() - datetime.timedelta(days=1)
D, d = before_one_day.strftime("%Y%m%d000000"), before_one_day.strftime("%Y%m%d235959")
db_date = d[2:8]
print(D, d, db_date)

20231025000000 20231025235959 231025


In [4]:
# conn = db_connection('')
# news_link = pd.read_sql(f"select * from db_name where is_status = 0 and daum IS NULL;", conn)
# conn.close()

In [ ]:
news_link = pd.read_xlsx(f'naver_news_{d}.xlsx')

In [5]:
# is_status = 0: 네이버 뉴스가 없다는 표기 -> 다음뉴스 수집
news_link = news_link[(news_link['is_status']==0)&(news_link['daum'].isnull()==True)]
news_link.head()

,id,ck_idx,news,title,href,is_status,text,daum,write_date,regist_date
0,3,84,광주매일신문,'2tv 생생정보 맛집'…홍게 무한리필 外,http://www.kjdaily.com/article.php?aid=1698226...,0,None,None,None,2023-10-25 23:42:23
1,5,84,금강일보,[생방송 오늘저녁] 쭈꾸미 볶음 맛집 위치는?,http://www.ggilbo.com/news/articleView.html?id...,0,None,None,None,2023-10-25 23:42:23
2,6,84,Queen,[생방송 투데이 맛집] '한우 더덕 불고기' 맛집,http://www.queen.co.kr/news/articleView.html?i...,0,None,None,None,2023-10-25 23:42:23
3,8,84,티브이데일리,[TD포토] 더보이즈 주연 '믿고보는 비주얼 맛집',http://www.tvdaily.co.kr/read.php3?aid=1698224...,0,None,None,None,2023-10-25 23:42:23
4,10,84,이투데이,"‘생방송 투데이’ 오늘방송맛집- 역주행, 눈과 입이 즐겁다! 파스타 맛집 ‘온○’",https://www.etoday.co.kr/news/view/2295934,0,None,None,None,2023-10-25 23:42:23


In [6]:
len(news_link)

8506

----

In [19]:
# 뉴스 링크 담을 DataFrame
news_href = pd.DataFrame(columns = ['ck_idx', 'news', 'title', 'href', 'is_status'])
daum_search = pd.DataFrame(columns = ['id', 'title', 'status'])

drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for i in tqdm(range(len(news_link))):
    crawl = True
    drv.get(f'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q={news_link["title"][i]}&sd={D}&ed={d}&period=u')
    time.sleep(2)
    soup = BeautifulSoup(drv.page_source, 'html.parser')
    
    news_box = soup.find('ul', {'class':'list_news'})
    try:
        news_list = news_box.findAll('div', {'class':'wrap_cont'})

        for lenth in range(len(news_list)):
            if len(news_list[lenth].find('span', {'class':'cont_info'}).findAll('a', {'class':'f_nb'})) >= 2:
                title = news_list[lenth].find('a', {'class':'tit_main fn_tit_u'}).get_text().strip()
                news_name = news_list[lenth].find('span', {'class':'cont_info'}).findAll('a', {'class':'f_nb'})[0].text.strip()
                href = news_list[lenth].find('span', {'class':'cont_info'}).findAll('a', {'class':'f_nb'})[-1]['href']

                data = pd.DataFrame({'ck_idx':news_link['ck_idx'][i], 'news':[news_name], 'title':[title], 'href':[href], 'is_status':[2]})
                news_href = pd.concat([news_href, data], ignore_index=True)
                
                data2 = pd.DataFrame({'id':news_link['id'][i], 'title':news_link['title'][i], 'status':[1]})
                daum_search = pd.concat([daum_search, data2], ignore_index=True)
            else:
                data2 = pd.DataFrame({'id':news_link['id'][i], 'title':news_link['title'][i], 'status':[0]})
                daum_search = pd.concat([daum_search, data2], ignore_index=True)
                pass
    except:
        data2 = pd.DataFrame({'id':news_link['id'][i], 'title':news_link['title'][i], 'status':[0]})
        daum_search = pd.concat([daum_search, data2], ignore_index=True)
        pass

drv.close()

100%|█████████████████████████████████████████| 400/400 [16:18<00:00,  2.45s/it]


In [20]:
print(len(news_href))
news_href

416


,ck_idx,news,title,href,is_status
0,62,뉴스1,"'슈퍼에이스' 복귀 지연, 전화위복되나…NC, 페디없이 스윕 노린다[준PO3]",http://v.daum.net/v/20231025081849225,2
1,62,한국일보,"한국 여자축구, 26일 태국과 올림픽 예선 첫 경기...반드시 승리해야",http://v.daum.net/v/20231025172253409,2
2,62,OSEN,"'0-3 패배' 라이언시티 감독 ""전북과 확연한 차이 인정""[전주톡톡]",http://v.daum.net/v/20231025211926796,2
3,62,스포츠서울,"가을 물들인 NC 젊은 마운드가 책임진다, ‘금빛’ 불펜 김영규의 존재감[SS인터뷰]",http://v.daum.net/v/20231025195907050,2
4,62,스포츠경향,"‘스트라이커 본능’ 손흥민, 케인 없어 더 훨훨 난다",http://v.daum.net/v/20231025154408201,2
...,...,...,...,...,...
411,62,OSEN,"""바르사 대신 2200억 사우디 선택 네이마르, 부상으로 바르사 복귀 불가능"" 西 매체",http://v.daum.net/v/20231025074820695,2
412,62,OSEN,"'차두리 코치 직관' 전북 vs 라이언시티, 선발 라인업 공개[오!쎈 전주]",http://v.daum.net/v/20231025184022004,2
413,62,마이데일리,"맨유, 반전 3연승→힘겹게 3경기 연속 1점 차 V, 부활 기틀 마련",http://v.daum.net/v/20231025114515114,2
414,62,스포츠경향,유일한 한국인 APC 직원 이승훈 매니저 “어릴 때 장애인스포츠를 자연스럽게 접할 ...,http://v.daum.net/v/20231025103620095,2


In [22]:
# 중복 제거
news_dup = news_href.drop_duplicates(['news', 'title'], ignore_index = True)
print(len(news_dup))

360


In [ ]:
# 저장
news_dup.to_xlsx(f'daum_news_{d}.xlsx')

In [23]:
# user_nm = db_id
# passwd = db_pw
# host_url = host
# port_num = num
# db_name = db_name
# engine = create_engine('')
# engine_conn = engine.connect()

# insert
# news_dup.to_sql(db_name, con=engine, if_exists='append', index=None)
# engine_conn.close()
# engine.dispose()

In [24]:
daum_search.drop_duplicates(['id', 'title'], ignore_index = True, inplace = True)
len(daum_search[daum_search['status']==1])

244

In [25]:
# for i in tqdm(range(len(daum_search))):
#     conn = db_connection('')
#     cur = conn.cursor()

#     cur.execute(
#         f'update db_name set daum={daum_search["status"][i]} where id = "{daum_search["id"][i]}"'
#     )
#     conn.commit()

#     cur.close()
#     conn.close()

100%|█████████████████████████████████████████| 400/400 [02:07<00:00,  3.13it/s]
